In [1]:
from balle import *
import matplotlib.pyplot as plt
import tensorflow_decision_forests as tfdf

2023-03-21 23:10:55.687141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 23:10:56.432251: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-21 23:10:56.432307: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-21 23:10:56.432315: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [18]:
trainer = train_mnist_model(lmbda=1000)
compressor, decompressor = make_mnist_codec(trainer)

Epoch 1/15
465/469 [============================>.] - ETA: 0s - loss: 159.3805 - distortion_loss: 0.0637 - rate_loss: 95.6470 - distortion_pass_through_loss: 0.0637 - rate_pass_through_loss: 95.6470

469/469 [==============================] - 8s 11ms/step - loss: 159.1602 - distortion_loss: 0.0636 - rate_loss: 95.5657 - distortion_pass_through_loss: 0.0636 - rate_pass_through_loss: 95.5602 - val_loss: 133.1790 - val_distortion_loss: 0.0477 - val_rate_loss: 85.4781 - val_distortion_pass_through_loss: 0.0477 - val_rate_pass_through_loss: 85.4804
Epoch 2/15
469/469 [==============================] - 5s 11ms/step - loss: 122.5249 - distortion_loss: 0.0468 - rate_loss: 75.7591 - distortion_pass_through_loss: 0.0468 - rate_pass_through_loss: 75.7541 - val_loss: 112.2437 - val_distortion_loss: 0.0471 - val_rate_loss: 65.1413 - val_distortion_pass_through_loss: 0.0471 - val_rate_pass_through_loss: 65.1484
Epoch 3/15
469/469 [==============================] - 5s 10ms/step - loss: 107.4754 - distortion_loss: 0.0471 - rate_loss: 60.3287 - distortion_pass_through_loss: 0.0471 - rate_pass_through_loss: 60.3256 - val_loss: 99.4346 - val_distortion_loss: 0.0505 - val_rate_loss: 48.9248 - val_dist

In [19]:
def get_latent(x,y):
    x = tf.cast(tf.expand_dims(x,axis=0),tf.float32) / 255.
    z = tf.round(compressor.analysis_transform(x)[0])
    z = tf.cast(z, tf.int8)
    return z,y

training_dataset2 = training_dataset.map(get_latent).batch(100)
validation_dataset2 = validation_dataset.map(get_latent).batch(100)

In [20]:
model = tfdf.keras.RandomForestModel(verbose=2)
model.fit(training_dataset2)

Use 12 thread(s) for training
Use /tmp/tmp89eou137 as temporary training directory
Reading training dataset...
Training tensor examples:
Features: Tensor("data:0", shape=(None, 50), dtype=int8)
Label: Tensor("data_1:0", shape=(None,), dtype=int64)
Weights: None
Normalized tensor features:
 {'data:0.0': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice:0' shape=(None,) dtype=float32>), 'data:0.1': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice_1:0' shape=(None,) dtype=float32>), 'data:0.2': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice_2:0' shape=(None,) dtype=float32>), 'data:0.3': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice_3:0' shape=(None,) dtype=float32>), 'data:0.4': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'strided_slice_4:0' shape=(None,) dtype=float32>), 'data:0.5': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>,

[INFO 2023-03-21T23:25:17.148445058-05:00 kernel.cc:756] Start Yggdrasil model training
[INFO 2023-03-21T23:25:17.148467299-05:00 kernel.cc:757] Collect training examples
[INFO 2023-03-21T23:25:17.148683407-05:00 kernel.cc:388] Number of batches: 600
[INFO 2023-03-21T23:25:17.148694298-05:00 kernel.cc:389] Number of examples: 60000
[INFO 2023-03-21T23:25:17.175253498-05:00 kernel.cc:774] Training dataset:
Number of records: 60000
Number of columns: 51

Number of columns by type:
	NUMERICAL: 50 (98.0392%)
	CATEGORICAL: 1 (1.96078%)

Columns:

NUMERICAL: 50 (98.0392%)
	1: "data:0.0" NUMERICAL mean:0 min:0 max:0 sd:0
	2: "data:0.1" NUMERICAL mean:0.0358833 min:-5 max:4 sd:1.25276
	3: "data:0.10" NUMERICAL mean:-0.0805 min:-5 max:3 sd:1.16285
	4: "data:0.11" NUMERICAL mean:-0.0216667 min:-2 max:2 sd:0.623081
	5: "data:0.12" NUMERICAL mean:0 min:0 max:0 sd:0
	6: "data:0.13" NUMERICAL mean:-0.0170167 min:-1 max:2 sd:0.342287
	7: "data:0.14" NUMERICAL mean:0 min:0 max:0 sd:0
	8: "data:0.15" N

Model trained in 0:00:20.698140
Compiling model...
Model compiled.


In [31]:
model.compile(metrics=["accuracy"])
model.evaluate(validation_dataset2)

100/100 [==============================] - 20s 194ms/step - loss: 0.0000e+00 - accuracy: 0.8913


[0.0, 0.8913000226020813]